In [1]:
from flask import Flask, request, jsonify
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
from joblib import load

try:
    cred = credentials.Certificate('./firebase-sdk.json')
    firebase_admin.initialize_app(cred)
except Exception as e:
    print("Firebase initialization failed:", e)

db = firestore.client()

# Initialize Flask app
app = Flask(__name__)

# Load label encoder
le_country = load('label_encoder.joblib')

feature_order = ['Name', 'Gender', 'Age', 'CountryofBirth', 'Smoking', 'Alcohol', 'FamilyHistory', 'CloseContact', 'HealthConditions', 'OccupationalRisks', 'PhysicalActivity', 'Diet', 'AirPollution', 'LongTermCough', 'ChestPain', 'AppetiteLoss', 'WeightLoss', 'Chills', 'Fatigue', 'NightSweat', 'CoughBlood', 'Fevers', 'NitricOxide']

# Load trained model
classifier = load('classifier.joblib')

# Function to fetch a user's data from Firestore
def fetch_user_data(user_id):
    doc_ref = db.collection('users').document(user_id)
    doc = doc_ref.get()
    if doc.exists:
        return doc.to_dict()
    else:
        print("No such document!")
        return None

# Function to preprocess user's data
def preprocess_user_data(user_data):
    df = pd.DataFrame([user_data])  # Convert dict to DataFrame
    df = df[feature_order]
    df['encoded_country'] = le_country.transform(df['CountryofBirth'])
    processed_df = df.drop(['Name', 'CountryofBirth'], axis=1)
    return processed_df

# Predict function for a user
def predict_tbpred(processed_data):
    prediction = classifier.predict(processed_data)
    return prediction

# Flask route to fetch user ID from Firestore
@app.route('/get-user-id', methods=['GET'])
def get_user_id():
    # Your logic to fetch user ID from Firestore goes here
    user_id = 'your_user_id'  # Replace with actual logic
    return jsonify({'userId': user_id})

# Flask route to predict tb_pred for a user
@app.route('/predict-tb', methods=['POST'])
def predict_tb():
    data = request.json
    user_id = data.get('userId')

    user_data = fetch_user_data(user_id)
    if user_data:
        processed_data = preprocess_user_data(user_data)
        prediction = predict_tbpred(processed_data)
        return jsonify({'prediction': prediction})
    else:
        return jsonify({'error': 'User data not found'}), 404

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\alice\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 1